In [27]:
# stage_cid10_joined
# Unifica todos os dados do CID10

In [28]:
%run ../spark-default.py

In [29]:
from delta.tables import DeltaTable

In [53]:
cid10_subcategorias = spark.table("stage.cid10_subcategorias")
cid10_subcategorias = cid10_subcategorias.withColumn("categoria", substring('sub_categoria', 0, 3))
cid10_subcategorias.show(truncate=False)

+-------------+-------------+----------------+-----------+----------------------------------------------------+-------------------------------------------------+----------+---------+---------+
|sub_categoria|classificacao|restringido_sexo|causa_obito|descricao                                           |descricao_abreviada                              |referencia|excluidos|categoria|
+-------------+-------------+----------------+-----------+----------------------------------------------------+-------------------------------------------------+----------+---------+---------+
|A000         |null         |null            |null       |Cólera devida a Vibrio cholerae 01, biótipo cholerae|A00.0 Colera dev Vibrio cholerae 01 biot cholerae|null      |null     |A00      |
|A001         |null         |null            |null       |Cólera devida a Vibrio cholerae 01, biótipo El Tor  |A00.1 Colera dev Vibrio cholerae 01 biot El Tor  |null      |null     |A00      |
|A009         |null         |null  

In [54]:
cid10_categorias = spark.table("stage.cid10_categorias")
cid10_categorias = cid10_categorias.withColumn("identificador_grupo_letra", substring('categoria', 0, 1)).withColumn("identificador_grupo_numero", substring('categoria', 2, 2).cast("integer"))
cid10_categorias.show(truncate=False)

+---------+-------------+---------------------------------------------------------------------------------+--------------------------------------------------+----------+-------------------------+--------------------------+
|categoria|classificacao|descricao                                                                        |descricao_abreviada                               |referencia|identificador_grupo_letra|identificador_grupo_numero|
+---------+-------------+---------------------------------------------------------------------------------+--------------------------------------------------+----------+-------------------------+--------------------------+
|A00      |null         |Cólera                                                                           |A00   Colera                                      |null      |A                        |0                         |
|A01      |null         |Febres tifóide e paratifóide                                                     |A

In [59]:
cid10_grupos = spark.table("stage.cid10_grupos") \
            .withColumn("categoria_inicio_letra", substring("categoria_inicio", 0, 1)) \
            .withColumn("categoria_inicio_numero", substring("categoria_inicio", 2, 2).cast("integer")) \
            .withColumn("categoria_fim_letra", substring("categoria_fim", 0, 1)) \
            .withColumn("categoria_fim_numero", substring("categoria_fim", 2, 2).cast("integer")) 

cid10_grupos.show(truncate=False)

+----------------+-------------+---------------------------------------------------------------------+--------------------------------------------------+----------------------+-----------------------+-------------------+--------------------+
|categoria_inicio|categoria_fim|descricao                                                            |descricao_abreviada                               |categoria_inicio_letra|categoria_inicio_numero|categoria_fim_letra|categoria_fim_numero|
+----------------+-------------+---------------------------------------------------------------------+--------------------------------------------------+----------------------+-----------------------+-------------------+--------------------+
|A00             |A09          |Doenças infecciosas intestinais                                      |Doenças infecciosas intestinais                   |A                     |0                      |A                  |9                   |
|A15             |A19          |

In [56]:
cid10_grupos.filter("substring(categoria_inicio, 0, 1) != substring(categoria_fim, 0, 1) ").show(30, truncate=False)

+----------------+-------------+-------------------------------------------------+-------------------------------------------------+----------------------+-----------------------+-------------------+--------------------+
|categoria_inicio|categoria_fim|descricao                                        |descricao_abreviada                              |categoria_inicio_letra|categoria_inicio_numero|categoria_fim_letra|categoria_fim_numero|
+----------------+-------------+-------------------------------------------------+-------------------------------------------------+----------------------+-----------------------+-------------------+--------------------+
|V01             |X59          |Acidentes                                        |Acidentes                                        |V                     |01                     |X                  |59                  |
|W00             |X59          |Outras causas externas de traumatismos acidentais|Outras causas externas de traumati

In [68]:
cid10_capitulos= spark.table("stage.cid10_capitulos") \
            .withColumn("categoria_inicio_letra", substring("categoria_inicio", 0, 1)) \
            .withColumn("categoria_inicio_numero", substring("categoria_inicio", 2, 2).cast("integer")) \
            .withColumn("categoria_fim_letra", substring("categoria_fim", 0, 1)) \
            .withColumn("categoria_fim_numero", substring("categoria_fim", 2, 2).cast("integer")) 

cid10_capitulos.show(30, truncate=False)

+---------------+----------------+-------------+--------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+----------------------+-----------------------+-------------------+--------------------+
|numero_capitulo|categoria_inicio|categoria_fim|descricao                                                                                                                 |descricao_abreviada                               |categoria_inicio_letra|categoria_inicio_numero|categoria_fim_letra|categoria_fim_numero|
+---------------+----------------+-------------+--------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------+----------------------+-----------------------+-------------------+--------------------+
|1              |A00             |B99          |Capítulo I - Alguma

In [81]:
cid10 = cid10_subcategorias.alias("sc") \
                .join(
                    cid10_categorias.alias("cat"),
                    col("sc.categoria") == col("cat.categoria"),
                    "left"
                ) \
                .join(
                    cid10_grupos.alias("g"),
                    (
                        (col("g.categoria_inicio_letra") == col("g.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") == col("g.categoria_inicio_letra"))
                        & (col("cat.identificador_grupo_numero") >= col("g.categoria_inicio_numero"))
                        & (col("cat.identificador_grupo_numero") <= col("g.categoria_fim_numero"))
                    )
                    | (
                        (col("g.categoria_inicio_letra") != col("g.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") == col("g.categoria_inicio_letra"))
                        & (col("cat.identificador_grupo_numero") >= col("g.categoria_inicio_numero"))
                    )
                    | (
                        (col("g.categoria_inicio_letra") != col("g.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") == col("g.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_numero") <= col("g.categoria_fim_numero"))
                    )
                    | (
                        (col("g.categoria_inicio_letra") != col("g.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") > col("g.categoria_inicio_letra"))
                        & (col("cat.identificador_grupo_letra") < col("g.categoria_fim_letra"))
                    )
                    ,
                    "left"
                ) \
                .join(
                    cid10_capitulos.alias("cap"),
                    (
                        (col("cap.categoria_inicio_letra") == col("cap.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") == col("cap.categoria_inicio_letra"))
                        & (col("cat.identificador_grupo_numero") >= col("cap.categoria_inicio_numero"))
                        & (col("cat.identificador_grupo_numero") <= col("cap.categoria_fim_numero"))
                    )
                    | (
                        (col("cap.categoria_inicio_letra") != col("cap.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") == col("cap.categoria_inicio_letra"))
                        & (col("cat.identificador_grupo_numero") >= col("cap.categoria_inicio_numero"))
                    )
                    | (
                        (col("cap.categoria_inicio_letra") != col("cap.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") == col("cap.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_numero") <= col("cap.categoria_fim_numero"))
                    )
                    | (
                        (col("cap.categoria_inicio_letra") != col("cap.categoria_fim_letra"))
                        & (col("cat.identificador_grupo_letra") > col("cap.categoria_inicio_letra"))
                        & (col("cat.identificador_grupo_letra") < col("cap.categoria_fim_letra"))
                    )
                    ,
                    "left"
                ) \
                .select(
                    col("sc.sub_categoria"),
                    col("sc.classificacao").alias("sub_categoria_classificacao"),
                    col("sc.restringido_sexo"),
                    col("sc.causa_obito"),
                    col("sc.descricao").alias("sub_categoria_descricao"),
                    col("sc.descricao_abreviada").alias("sub_categoria_descricao_abreviada"),
                    col("sc.referencia").alias("sub_categoria_referencia"),
                    col("sc.excluidos").alias("sub_categoria_excluidos"),
                    col("cat.categoria").alias("categoria"),
                    col("cat.classificacao").alias("categoria_classificacao"),
                    col("cat.descricao").alias("categoria_descricao"),
                    col("cat.descricao_abreviada").alias("categoria_descricao_abreviada"),
                    col("cat.referencia").alias("categoria_referencia"),                    
                    col("cap.numero_capitulo").alias("capitulo_numero"),
                    col("cap.descricao").alias("capitulo_descricao"),
                    col("cap.descricao_abreviada").alias("capitulo_descricao_abreviada")
                )


cid10.show(truncate=False)

+-------------+---------------------------+----------------+-----------+----------------------------------------------------+-------------------------------------------------+------------------------+-----------------------+---------+-----------------------+----------------------------------------+------------------------------------+--------------------+---------------+-------------------------------------------------------+-----------------------------------------------+
|sub_categoria|sub_categoria_classificacao|restringido_sexo|causa_obito|sub_categoria_descricao                             |sub_categoria_descricao_abreviada                |sub_categoria_referencia|sub_categoria_excluidos|categoria|categoria_classificacao|categoria_descricao                     |categoria_descricao_abreviada       |categoria_referencia|capitulo_numero|capitulo_descricao                                     |capitulo_descricao_abreviada                   |
+-------------+---------------------------+-

In [91]:
cid10.count()

18572

In [92]:
cid10.distinct().count()

12451

In [93]:
cid10_subcategorias.count()

12451

In [89]:
cid10.groupBy("sub_categoria").agg(count("*").alias("count"), collect_list(col("categoria")), collect_list(col("capitulo_numero"))).filter("count > 1").show(truncate=False)

+-------------+-----+-----------------------+-----------------------------+
|sub_categoria|count|collect_list(categoria)|collect_list(capitulo_numero)|
+-------------+-----+-----------------------+-----------------------------+
|C000         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C001         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C002         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C003         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C004         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C005         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C006         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C008         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C009         |3    |[C00, C00, C00]        |[2, 2, 2]                    |
|C01          |3    |[C01, C01, C01]        |[2, 2, 2]                    |
|C020       

In [80]:
cid10.first().asDict()

{'sub_categoria': 'A000',
 'sub_categoria_classificacao': None,
 'restringido_sexo': None,
 'causa_obito': None,
 'sub_categoria_descricao': 'Cólera devida a Vibrio cholerae 01, biótipo cholerae',
 'sub_categoria_descricao_abreviada': 'A00.0 Colera dev Vibrio cholerae 01 biot cholerae',
 'sub_categoria_referencia': None,
 'sub_categoria_excluidos': None,
 'categoria': 'A00',
 'categoria_classificacao': None,
 'categoria_descricao': 'Cólera',
 'categoria_descricao_abreviada': 'A00   Colera',
 'categoria_referencia': None,
 'identificador_grupo_letra': 'A',
 'identificador_grupo_numero': 0,
 'capitulo_numero': '1',
 'capitulo_descricao': 'Capítulo I - Algumas doenças infecciosas e parasitárias',
 'capitulo_descricao_abreviada': 'I.   Algumas doenças infecciosas e parasitárias'}

In [83]:
DeltaTable.createIfNotExists(spark) \
  .tableName("stage.cid10_joined") \
  .addColumns(cid10.schema) \
  .execute()

In [94]:
cid10.distinct().write.insertInto("stage.cid10_joined")

In [95]:
spark.stop()